## **Policy Gradient**

Up untill now, we focused on determining the state-action value function Q. The policy was only a subproduct of Q. \
We might wonder whether it is possible to approximate the optimal policy **directly**.
The answer is Policy Gradient.

### **Theory**

#### Policy Gradient Theorem

Let G be the average return. Let $ \pi{_\theta}$ be a parametrized policy. Then we have that: 
$$
\nabla_\theta G = \sum_{s a} \pi(a|s)\,\eta_\pi(s)\,Q(s, a)\; \nabla_\theta \log(\pi(a|s)) 
$$
The powerful aspect of this equation is this: the summatory is obtained expermentally. The algorithm needs only to consider $Q(s, a)\; \nabla_\theta \log(\pi(a|s))$ for every step. In fact we might rewrite the same equation as follows: 
$$
\nabla_\theta G = E_\pi \left[ \sum_{t = 0}^{\infty} Q(S_t, A_t)\; \nabla_\theta \log(\pi(A_t|S_t))\qquad | \qquad S_0 \sim \rho_0 \; A \sim \pi \right]\qquad (*)
$$

From now on we will use the **Softmax parametrization** in a tabular contex. 


#### Actor Only

We can use the equation (*) directly. Since the policy must remain fixed as we calculate the gradient, we update at the end of each episode. Thus, we can use a Monte Carlo estimator for $Q$. Here we show a simple pseudocode:

collect  $\{S_t, A_t, R_t\}_{t= 0, 1, ..., T}$ 

$G = 0$ \
for $t = T, T-1, ... 0$ \
$\qquad G = R_t + \gamma G$ \
$\qquad \theta = \theta +  \alpha G \nabla_\theta \log(\pi(A_t | S_t))$

Here $\alpha$ is a fixed learning rate. In the case of softmax, we have that $\theta = \{ \theta_{sa}\} \text{ with } s \in S \; a \in A$. Then, the gradient is 
$$
\nabla_\theta \log(\pi(A_t | S_t))_{\theta_{s,a}} = 
\begin{cases}
1 - \pi(A_t | S_t)\qquad &s, a = S_t, A_t \\
-\pi(A_t | S_t) \qquad &s = S_t \; a \neq A_t \\
0 \qquad &\text{else}
\end {cases}
$$

There is an intuitive way to interpret this code: if, in a certain state $s$, an action $a$ yielded good results, we increase $\theta_{s, a}$ and decrease $\theta$ of the other actions in the same state.


#### Actor Critic

Actor Only is a very simple, but not efficient approach. The main problem is that we value an action through the return obtained afterwards, but *without a contex*. A same return can be very suboptimal, if the sistem is generous, or very good, if rewards are rare. A better and equivalent way to estimate $\nabla_\theta G$ is through the *advantage*, defined as:
$$
Ad(s, a) = Q(s, a) - V(s)
$$

If we use bootstrapping we can write 
$$
Ad(s, a) = R_t + \gamma V(S_{t+1}) - V(S_t) = \delta_t
$$

And we already have a way to estimate $V(s)$: Temporal Different Learning. We can combine the two, obtaining what is called Actor Critic. In fact we have a Critic, which estimates $V$, and an Actor, which estimates $\pi$ through gradient ascent. We present here a simple pseudocode:

collect $S_t, A_t, R_t, S_{t+1}$ 

$\delta = R_t + \gamma V(S_{t+1}) - V(S_t)$ \
$V(S_t) = V(S_t) + \alpha_V \delta $

$\theta = \theta +  \alpha_A \delta \nabla_\theta \log(\pi(A_t | S_t))$

For the learning rates, we ask that $\alpha_V > \alpha_A$. In fact the Actor is using the Critic for its estimation, thus the Critic must learn faster than the Actor.


#### General Advantage Estimation

With Actor Critic algorithm we have to update the policy at every step. This can be time consuming. Furthermore, it could be better to update $\pi$ only at episode ended, so that we can have a better idea on the long term performance of the policy. For that we use an algorithm called General Advantage Estimation (GAE).

So far we estimated $Ad$ with only the current reward. But, since we update at the end of the episode, we can actually consider $n$ future rewards, obtaining:
$$

Ad_t^n = R_t + \gamma R_{t+1} + ... + \gamma^n R_{t + n} + \gamma^{n+1} V(S_{t+n+1}) \;- \; V(S_t) = [R_t -  V(S_t)] \; +\; \gamma R_{t+1} + ... + \gamma^n R_{t + n} + \gamma^{n+1} V(S_{t+n+1}) \\
       = \delta_t + \gamma[ R_{t+1} - V(S_{t+1})] + ... + \gamma^n R_{t + n} + \gamma^{n+1} V(S_{t+n+1}) = ... = \delta_t + \delta_{t+1} + ... + \delta_{t+n}

$$
The idea behind GAE is to consider $Ad^n$ for all $n$, each discounted by $\lambda^n$. Specifically 
$$
Ad_t^{GAE} = (1-\lambda) \sum_{n = 0}^{\infty} \lambda^nAd_t^n = (1-\lambda) \left[ \delta_t(1 + \lambda + \lambda^2 + ...) + \gamma \delta_{t+1}(\lambda + \lambda^2 + ...) + ...\right] = \sum_{n = 0}^{\infty} (\lambda\gamma)^n\delta_{t+n} 
$$

The advantage can be seen also in another way. If we conider the average of $Ad$ with fixed state-action, we obtain the advantage. Instaed, if we consider the average of $Ad$ with a fixed state, we obtain a measure of how much owr estimator is correct. Indeed we will use $Ad$ both for estimating $V$ and $\pi$. Here we present a pseudocode: 

collect  $\{S_t, A_t, R_t\}_{t= 0, 1, ..., T}$ 

$Ad = 0$ \
for $t = T, T-1, ... 0$ \
$\qquad \delta = R_t + \gamma V(S_{t+1}) - V(S_t) $\
$\qquad Ad = \delta_t + \lambda\gamma Ad$ 

$\qquad V(S_t) = V(S_t) + \alpha_V\, Ad$

$\qquad \theta = \theta +  \alpha_A\; Ad \;\nabla_\theta \log(\pi(A_t | S_t))$

With this algorithm we have an interesting variabile in hand: $\lambda$. If $\lambda$ is close to 0, we learn only from close rewards. Thus we will have less variance, but there will be a bias due to bootstrapping. If, on the contrary, $\lambda$ is close to 1, distance rewards have a great influence. Thus variance will increase, but there will be less bias. In fact we are moving toward a Monte Carlo apprach, notoriously unbiased.

A little note: the algorithm presented here is actually a variation of the standard GAE. In fact we are calculating the advantages, and at the same time updating the values. Usually the advantages are calculated first, and then the values are updated. This is clearly a convenient approach for batch training with Neural Networks. Here we are not using NNs, and then we get to choose one of the two approaches. The "GAE in place" algorithm has been chosen because it exhibits lower variance. Infact $\delta_t$ is calculated with a value function already updated.

### **Implementation**

As already discussed, the dimentionality of the problem forces us to use binnings. For the rest of the text, we will improperly call state what is actually a bin.

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

All the algorithms developed in this project are subclasses of one general class. In it we define a method called "learning", which is the backbone of all the algorithms. Since its implementation presents some technical details, we show simply a pseudocode:

for each episode:

$\qquad$ initialize $S_0$\
$\qquad$ take $A_0 \sim \pi(\,\cdot\,|\,S_0\,)$ --> here we call "get_action_during_learning(state)"\
$\qquad$ loop over t untill termination:

$\qquad \qquad$ observe $S_{t + 1} \; R_{t + 1}$\
$\qquad \qquad$ append $(S_t, A_t, R_{t+1})$ to episode\
$\qquad \qquad$ take $A_{t+1} \sim \pi(\,\cdot\,|\,S_{t+1}\,)$ --> here we call "get_action_during_learning(state)"\
$\qquad \qquad$ if needed, do step update --> here we call "single_step_update(S_t, A_t, R_{t+1}, S_{t+1}, A_{t+1}, ...)"

$\qquad$ if needed, do episode update --> here we call "single_episode_update(episode)"
    
Let us dive deeper into the implementation. Here we have some common elements used in the algorithms:\
    1) "parameters" is a numpy array containing $\theta_{sa}$ for all state-action pairs.\
    2) If used, "value" is a numpy array containing $V(s)$ for all states.\
    3) "index_sa" is a utility that associates a state-action pair to an index for the arrays.\
    4) "index_s" is a utility that associates a state to an index for the arrays.\
    5) "policy" is a utility that takes as input an action and a state, performes the softmax, and returns the probability of taking that action, given that we are in the inserted state.

Of these five elements, only the last one is theoretically interesting, thus we show its implementation:

In [1]:
def policy(self, action, state):
        # This functions performes the Softmax, obtaining prob(A = action | S = state ) = exp( parameters(s, a) )  /  (normaliz. factor over actions)

        return np.exp(self.parameters[self.index_sa((*state, action))])  /  sum(  np.exp(self.parameters[self.index_sa((*state, a))])  for a in range(self.action_space)  )

With this last utility we can implement the method "get_action_during_learning" as follows:

In [2]:
def get_action_during_learning(self, state, possible_actions=None):     
    
    prob_a = np.zeros(self.action_space)

    for a in range(self.action_space):
        prob_a[a] = self.policy(a, state)       #we construct the probability distribution of actions following the policy in the current state 
    
    chosen_action = np.random.choice(range(self.action_space), p=prob_a)        #we choose an action following the distribution "prob_a"
    return chosen_action
        


We still need to see the realization of "single_step_update" and "single_episode_update". For that we need to analyze each algorithm separately.

#### Actor Only

This strategy has to be considered just as an intermediate step. In fact it is too simplistic and doesn't have clear advantages. Nonetheless we show it for completeness. As seen before, we need only to update the policy at the end of the episode.

In [3]:

def single_episode_update(self, episode):
    # After the episode ended, we update the policy
    # The policy is parametrized via soft-max, and theta is a vector with entries for every pair state-action
    G = 0
    for state, action, reward in reversed(episode):
        G = reward + self.gamma * G

        # Update the parameters
        #                                       1 - pi(a|s)   if a is the performed action
        # we recall that grad log(pi(a|s)) = 
        #                                       - pi(a|s)     else

        for a in range(self.action_space):
            if a == action:  #for the performed action
                self.parameters[self.index_sa((*state, a))] += self.lr_a*G*(1-self.policy(action, state))
            else:
                self.parameters[self.index_sa((*state, a))] += self.lr_a*G*( -self.policy(action, state))

        #for numerical stability, we subtract from the parameters their max value
        max_parameter = np.max( [self.parameters[self.index_sa((*state, a))]   for a in range(self.action_space)])
        for a in range(self.action_space):
            self.parameters[self.index_sa((*state, a))] -= float(max_parameter)

At the end of the function we see a technical detail. If we analyze the function "policy", we notice that we calculate the exponential of the parameters. If these numbers are too large, we encounter numerical problems. On the other hand, we don't find this issue for large negative numbers. For dealing with such a problem, we can use the following propiety of the Softmax:
$$
\frac{e^{\theta}}{\sum_{\theta'} e^{\theta'}} = \frac{e^{\theta -M}}{\sum_{\theta'} e^{\theta'-M}} \qquad \forall M \in \mathbb{R}
$$
Since we want to minimize the parameters, we subtract their maximum every time we modify them.

#### Actor Critic

For this algorithm we update at every step, thus we override only "single_step_update":

In [4]:
def single_step_update(self, s, a, r, new_s, new_a, done):
    # After the episode ended, we update the value and the policy
    # The policy is parametrized via soft-max, and theta is a vector with entries for all pair state-action

    # critic update
    if done:   
        delta = r + 0 - self.value[self.index_s(s)]
    else:
        delta = r + self.gamma*self.value[self.index_s(new_s)]    -    self.value[self.index_s(s)]

    self.value[self.index_s(s)] += self.lr_v*delta

    # actor update (parameters)
    #                                       1 - pi(a|s)   if a is the performed action
    # we recall that grad log(pi(a|s)) = 
    #                                       - pi(a|s)     else
    for ap in range(self.action_space):
        if ap == a:  #for the performed action
            self.parameters[self.index_sa((*s, ap))] += self.lr_a*delta*(1-self.policy(a, s))
        else:
            self.parameters[self.index_sa((*s, ap))] += self.lr_a*delta*( -self.policy(a, s))

    # For numerical stability, we subtract from the parameters their max value
    max_parameter = np.max( [self.parameters[self.index_sa((*s, ap))]   for ap in range(self.action_space)])
    for ap in range(self.action_space):
        self.parameters[self.index_sa((*s, ap))] -= float(max_parameter)
 

We note that we have:
$$
\alpha_A = \text{lr\_a}\\
\alpha_V = \text{lr\_v}
$$
#### GAE

For GAE we update the policy and value function only at the end of each episode:

In [5]:
def single_episode_update(self, episode):
    # After the episode ended, we update the value and the policy
    # The policy is parametrized via soft-max, and theta is a vector with entries for every couple bin-action

    GAE = 0       # GAE is a variable used to store the Generalized Advantage Estimator for all t 

    for t in range(len(episode) -1, -1, -1):
        
        state, action, reward = episode[t]

        if t != len(episode) -1:
            next_state = episode[t+1][0]

            delta = reward + self.gamma*self.value[self.index_s(next_state)] - self.value[self.index_s(state)]
        else:

            delta = reward - self.value[self.index_s(state)]

        GAE = delta + self.gamma*self.Lambda*GAE        #GAE is overwritten at each step, and we have GAE = GAE(t)
        
        #update the value

        self.value[self.index_s(state)] += self.lr_v*GAE

        # Update the parameters
        #                                       1 - pi(a|s)   if a is the performed action
        # we recall that grad log(pi(a|s)) = 
        #                                       - pi(a|s)     else

        for a in range(self.action_space):
            if a == action:  #for the performed action
                self.parameters[self.index_sa((*state, a))] += self.lr_a*GAE*(1-self.policy(action, state))
            else:
                self.parameters[self.index_sa((*state, a))] += self.lr_a*GAE*( -self.policy(action, state))
            

        #for numerical stability, we subtract from the parameters their max value
        
        max_parameter = np.max( [self.parameters[self.index_sa((*state, a))]   for a in range(self.action_space)])
        for a in range(self.action_space):
            self.parameters[self.index_sa((*state, a))] -= float(max_parameter)

We note that, also here, we have:
$$
\alpha_A = \text{lr\_a}\\
\alpha_V = \text{lr\_v}
$$
### **Results**

For all the results shown in this chapter, we used as bin the 8 directions of food (sud, sud-est, est, ...) and a neighborhood of the snake's head. The total number of bins can be calculated in this way: 
$$
N = n_{\text{food directions}} \cdot 2^{\text{size neighborhood}} = 8 \cdot 2^{\text{size neighborhood}}\qquad (*)
$$

As a target return, we have built a simple reference policy. It sais: don't go against a block, if possible go towards food, else go randomly in a free direction. For all examples, unless stated otherwise, we set reward food = 20, penality step = 0.2, penality death = 10. With this data, the average performance of this policy is around 400. 

#### Using Von Neuemann neighborhood
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
For this part, we have learned policies using as bin the Von Neuemann radius 1 neighborhood plus the direction of food. This binning has been the most successfull one, since the number of different bins is small enough for a thorough learning. Specifically, using (*) the total number of bins is 128. 

Let us start with the most successfull learining: Actor Critic with lr_v = 0.2, lr_a = 0.01. With such a little knowledge of the state, the reference policy we built might seem as the optimal one. But here the surprise:

<img src="images/V1_0.2_0.01_-_(20,-0.2,-10).png" width=600>

As we can see, owr algorithm finds even a better policy. In a nushell, it learns how to coil up, in order to reduce the risk of getting blocked within its own tail. 

This result was possible thanks to a careful tuning of the paramters. For example, if the learning rates are too high, the policy convinces itself of wrong strategies:

![degenerated policy](images/lr_troppo_alto_AC_0.9_0.09_-_(20,-0.2,-10).png)
<img src="images/V1_0.9_0.09_-_(20,-0.2,-10).png" width=400>


Here we can see that:\
    -the food is est\
    -there are blocks to the north and west\
    -the policy is collapsed and gives with probability 1 the action: go north

This example was performed with Actor Critic, lr_v = 0.9, lr_a = 0.09. The learining rates are indeed too large, and the learning is killed.

Also GAE performes well in this contex, for any value of $\lambda$:

<img src="images/V1_0.1_0.005_0.2_(20,-0.2,-10).png" width=400>

<img src="images/V1_0.05_0.003_0.9_(20,-0.2,-10).png" width=400>

We can see that for this binning bootstrapping is a good idea. In fact, since the states aren't much, the knowledge of them is valuable.

The last two plots give us the opportunity to point out that the laerning rates must be smaller for a larger $\lambda$. This comes from the fact that $\lambda$ increases variance, as discussed before. If, instead, we keep the same learning rates, the training is too unstable:




$$
\text{Actor Critic: } \alpha_V = 0.9 \quad \alpha_A = 0.09
$$

$$
\text{GAE : }\, \alpha_V = 0.1 \quad \alpha_A = 0.005 \quad \lambda = 0.9
$$


